## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.30"

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")

cursor = conn.cursor()


## Code to Add All Subdirs of "Python_scripts" to sys.path

In [ ]:
import sys
from pathlib import Path

# Get root directory where Jupyter Lab starts
root_dir = Path().resolve()
scripts_dir = root_dir / "Python_scripts"

# Add all subdirectories of Python_scripts to sys.path
for subdir in scripts_dir.rglob("*"):
    if subdir.is_dir():
        sys.path.append(str(subdir))


## Insert video file names

In [ ]:
from pathlib import Path

# Define base folder
base_folder = Path("data/FoodLight/SplitVideos")

# Collect all .mp4 and .avi filenames (non-recursive or recursive)
video_files = list(base_folder.rglob("*.mp4")) + list(base_folder.rglob("*.avi"))

# Convert to relative path strings
video_records = [(str(vf.relative_to(base_folder.parent)),) for vf in video_files]

# Insert into dlc_table
insert_query = "INSERT INTO dlc_table (video_name) VALUES (%s);"

# Avoid Duplicate Inserts
cursor.execute("SELECT video_name FROM dlc_table;")
existing = set(row[0] for row in cursor.fetchall())

# Filter only new videos
new_records = [vr for vr in video_records if vr[0] not in existing]

cursor.executemany(insert_query, new_records)
conn.commit()


## Insert task, date_str, name, health, id_ in dlc_table

In [ ]:
from parse_video_name import parse_video_name

cursor.execute("SELECT id, video_name FROM dlc_table WHERE video_name IS NOT NULL;")
rows = cursor.fetchall()

updates = []

for row in rows:
    id_, video_name = row
    task, date_str, name, health = parse_video_name(video_name)
    updates.append((task, date_str, name, health, id_))

# Update in batch
cursor.executemany("""
UPDATE dlc_table
SET task = %s, date = %s, name = %s, health = %s
WHERE id = %s;
""", updates)

conn.commit()


### Completely Clear Table First (CAREFUL)

In [ ]:
cursor.execute("DELETE FROM dlc_files")
conn.commit()
print("🗑️ All entries deleted from dlc_files.")
# conn.rollback()

## Insert csv files into columns

In [ ]:
# Folder with new CSVs
csv_dir = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Insert each CSV path
for csv_file in csv_files:
    coord_path = os.path.join(csv_dir, csv_file)

    # Optional: check if already in DB to avoid duplicates
    cursor.execute("SELECT 1 FROM dlc_files WHERE coord_path = %s", (coord_path,))
    exists = cursor.fetchone()
    if exists:
        print(f"⚠️ Already in DB: {csv_file}")
        continue

    cursor.execute("INSERT INTO dlc_files (coord_path) VALUES (%s)", (coord_path,))
    print(f"✅ Inserted: {csv_file}")

conn.commit()
# cursor.close()
print("🎉 All new coord_paths uploaded with video_path = NULL.")


## Show first few columns of the dlc_files table

In [ ]:
# Read data into DataFrame
df = pd.read_sql_query("SELECT * FROM dlc_files LIMIT 5", conn)

# Optional: show full column content
pd.set_option('display.max_colwidth', None)

# Show first 5 entries
print(df)

conn.close()


## Add path name

In [ ]:
import importlib
import sys
sys.path.append('../Python_scripts')

import update_dlc_table
importlib.reload(update_dlc_table)

## Insert animal names

In [ ]:
update_dlc_table.update_column("name")

## Fix bad entries

In [ ]:
import psycopg2

fixes = {
    'Eli.': 'Eli',
    'Orelans': 'NewOrleans',
    'London.': 'London'
}

conn = psycopg2.connect(
    dbname="deeplabcut_db",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

for old_name, corrected_name in fixes.items():
    cursor.execute(
        "UPDATE dlc_files SET name = %s WHERE name = %s",
        (corrected_name, old_name)
    )
    print(f"✔ Updated '{old_name}' → '{corrected_name}'")

conn.commit()
cursor.close()
conn.close()
print("🎉 Name cleanup done.")

## Insert maze number

In [ ]:
update_dlc_table.update_column("maze")

## Insert task name

In [ ]:
update_dlc_table.update_column("task")

## Insert health name

In [ ]:
update_dlc_table.update_column("health")

In [ ]:
update_dlc_table.update_column("stop_count")